## Classification Trees and Random Forests

### Classification Trees

In [1]:
from __future__ import division
from scipy.stats import mode
import pandas as pd
import numpy as np

#### Problem 1

In [2]:
def gini(labels):
    return 1 - np.sum((np.bincount(labels)/len(labels))**2)

#### Problem 2

In [3]:
def split(D, L, p, x):
    d1, d2 = D[:,p]<=x, D[:,p]>x
    return D[d1], L[d1], D[d2], L[d2]

#### Problem 3

In [4]:
def info_gain(D, L, p, x):
    _, L1, _, L2 = split(D, L, p, x)
    return gini(L) - (len(L1)*gini(L1) + len(L2)*gini(L2))/len(L)

#### Problem 4

In [5]:
def best_split(D, L):
    best, best_loc = 0, None
    for p in xrange(D.shape[1]):
        vals = set(D[:,p])
        for x in vals:
            curr = info_gain(D, L, p, x)
            if best <= curr:
                best = curr
                best_loc = (p,x)
    return best_loc

#### Problems 5, 6, 7

In [9]:
class Node(object):
    
    def __init__(self, tree, D, L, depth, max_depth, tol):
        tree.nodes.append(self)
        self.depth = depth
        self.score = gini(L)
        
        if self.depth >= max_depth or self.score < tol:
            self.label = np.argmax(np.bincount(L)) if len(L) != 0 else None
            self.leaf = True
        else:
            self.p, self.x = best_split(D, L)
            D1, L1, D2, L2 = split(D, L, self.p, self.x)
            self.left = Node(tree, D1, L1, depth+1, max_depth, tol)
            self.right = Node(tree, D2, L2, depth+1, max_depth, tol)
            self.leaf = False
    
    def predict(self, sample):
        if self.leaf:
            return self.label
        elif sample[self.p] <= self.x:
            return self.left.predict(sample)
        else:
            return self.right.predict(sample)
    def __str__(self):
        out = "Leaf Node," if self.leaf else "Parent Node,"
        out += " Gini: {},".format(self.score)
        out += " Depth: {},".format(self.depth)
        if self.leaf:
            out += " Label: {},".format(self.label)
        else:
            out += " split index (p): {},".format(self.p)
            out += " split threshold (x): {},".format(self.x)
        return out

class ClassificationTree(object):
    
    def __init__(self, D, L, max_depth=10, tol=.2):
        self.max_depth = max_depth
        self.nodes = []
        self.root = Node(self, D, L, 1, max_depth, tol)

    def accuracy(self, data, labels):
        return 100*np.mean(self.classify(data) == labels)
    
    def classify(self, data):
        return np.array([self.predict(d) for d in data])
    
    def predict(self, sample):
        return self.root.predict(sample)
    
    def __str__(self):
        return "\n\n".join([str(node) for node in self.nodes])

In [10]:
def process_titanic_data(filename="titanic.csv", pclass_change=False):
    data = pd.read_csv(filename)
    data = data[["Survived", "Pclass", "Sex", "Age"]]
    data.dropna(inplace=True)
    data.loc[data.Sex=="female", "Sex"] = 0.
    data.loc[data.Sex=="male", "Sex"] = 1.
    
    if pclass_change is True:
        pclass = data["Pclass"]
        data["SecondClass"] = [1 if 2==pclass[i] else 0 for i in pclass.index]
        data["FirstClass"] = [1 if 1==pclass[i] else 0 for i in pclass.index]
        data.drop("Pclass", axis=1, inplace=True)
    
    test_index = sorted(np.random.choice(data.index, int(len(data.index)*.4), replace=False))
    train_index = [i for i in data.index if i not in test_index]
    
    return data.loc[train_index], data.loc[test_index]

In [11]:
training, testing = process_titanic_data()

test_data = np.array(testing.drop("Survived", axis=1), dtype=np.float)
test_labels = np.array(testing["Survived"], dtype=np.int64)
train_data = np.array(training.drop("Survived", axis=1), dtype=np.float)
train_labels = np.array(training["Survived"], np.int64)

tree = ClassificationTree(train_data, train_labels, max_depth=8, tol=.1)
print(tree)

print "\n{}% Correct".format(tree.accuracy(test_data, test_labels))

Parent Node, Gini: 0.490182157491, Depth: 1, split index (p): 1, split threshold (x): 0.0,

Parent Node, Gini: 0.381223077466, Depth: 2, split index (p): 0, split threshold (x): 2.0,

Parent Node, Gini: 0.141107253086, Depth: 3, split index (p): 0, split threshold (x): 1.0,

Leaf Node, Gini: 0.0505987518975, Depth: 4, Label: 1,

Parent Node, Gini: 0.23256850078, Depth: 4, split index (p): 2, split threshold (x): 50.0,

Parent Node, Gini: 0.213039485767, Depth: 5, split index (p): 2, split threshold (x): 17.0,

Leaf Node, Gini: 0.0, Depth: 6, Label: 1,

Parent Node, Gini: 0.260355029586, Depth: 6, split index (p): 2, split threshold (x): 44.0,

Parent Node, Gini: 0.292345679012, Depth: 7, split index (p): 2, split threshold (x): 42.0,

Leaf Node, Gini: 0.267561983471, Depth: 8, Label: 1,

Leaf Node, Gini: 0.0, Depth: 8, Label: 0,

Leaf Node, Gini: 0.0, Depth: 7, Label: 1,

Leaf Node, Gini: 0.0, Depth: 5, Label: 0,

Parent Node, Gini: 0.497962879131, Depth: 3, split index (p): 2, split t

### Random Forests

#### Problem 1, 2

In [12]:
class ForestNode(object):
    
    def __init__(self, D, L, features, depth, max_features, max_depth, tol):
        self.depth = depth
        self.score = gini(L)
        
        # Base Case: max depth or impurity tolerance reached.
        if self.depth >= max_depth or self.score < tol or len(features) < max_features:
            self.label = mode(L)[0][0] if len(L) != 0 else -1
            self.leaf = True
        
        # Recursive case: split the data.
        else:
            self.p, self.x = self._best_split(D, L, features)
            D1, L1, D2, L2 = split(D, L, self.p, self.x)
            if len(set(D1[:,self.p])) == 1 or len(set(D2[:,self.p])) == 1:
                features.remove(self.p)
            
            self.left = ForestNode(D1, L1, features, depth+1, max_features, max_depth, tol)
            self.right = ForestNode(D2, L2, features, depth+1, max_features, max_depth, tol)
            self.leaf = False
    
    @staticmethod
    def _best_split(D, L, pvalues):
        best, best_loc = 0, None
        for p in pvalues:
            vals = set(D[:,p])
            for x in vals:
                curr = info_gain(D, L, p, x)
                if best <= curr:
                    best = curr
                    best_loc = (p,x)
        return best_loc
    
    def predict(self, sample):
        if self.leaf:
            return self.label
        elif sample[self.p] <= self.x:
            return self.left.predict(sample)
        else:
            return self.right.predict(sample)

class ForestTree(object):
    
    def __init__(self, D, L, features, max_depth=10, tol=.2):
        self.root = ForestNode(D, L, features, 1, len(features), max_depth, tol)
    
    def predict(self, sample):
        return self.root.predict(sample)

#### Problem 3, 4

In [13]:
class RandomForest(object):
    
    def __init__(self, D, L, num_trees, num_features, max_depth, tol):
        self.trees = []
        for i in xrange(num_trees):
            features = sorted(list(np.random.choice(np.arange(D.shape[1]), num_features, replace=False)))
            self.trees.append(ForestTree(D, L, features, max_depth, tol))
    
    def accuracy(self, data, labels):
        return 100*np.mean(self.classify(data) == labels)
    
    def classify(self, data):
        return np.array([self.predict(d) for d in data])
    
    def predict(self, sample):
        return mode([t.predict(sample) for t in self.trees])[0][0]

#### Problem 5

In [14]:
training, testing = process_titanic_data()
test_data = np.array(testing.drop("Survived", axis=1), dtype=np.float)
test_labels = np.array(testing["Survived"], dtype=np.int64)
train_data = np.array(training.drop("Survived", axis=1), dtype=np.float)
train_labels = np.array(training["Survived"], np.int64)
tree = RandomForest(train_data, train_labels, 100, 2, 8, .1)

print "{}% Correct".format(tree.accuracy(test_data, test_labels))

74.1626794258% Correct
